# Cleaning the Trip Files

There are a huge amount of trip observations. I make it at about 30,000,000. The guy in the video said a couple million. I can't guarantee my count or his at the moment. The point is that they should perhaps be kept in separate files, but with consistent file and column names.

In [2]:
from cs1.tools import *

In [3]:
pwd()

'/home/fuzzy/DataAnalysisLab/Case_Study_1'

Since I'm more familiar with `pandas` than any other data manipulation tool, it's the one I'm starting with. My goal is to get all of the data into a `mysql` database and, preferably, to have the table for the trips partitioned by month (and year). The first hurdle to overcome, of course, is the inconsistency in the column names. I'm sure there's some history as to when and why the names were changed, but none of that is really relevant. The main point is just to make sure that they are consistent within the database. The most direct way to do that is to make them consistent in their CSV files.

The `unique-trip_cols` function prints all of the column names found in all of the files.

In [5]:
columnize(unique_trip_cols())

start_station_name              start_time                                      
trip_id                         starttime                                       
"end_time"                      end_lng                                         
"bikeid"                        start_lat                                       
bikeid                          "gender"                                        
ended_at                        05 - Member Details Member Birthday Year        
from_station_id                 end_station_name                                
usertype                        ride_id                                         
gender                          02 - Rental End Station Name                    
03 - Rental Start Station ID    "birthyear"                                     
start_lng                       end_time                                        
03 - Rental Start Station Name  member_casual                                   
tripduration                

In [4]:
consist_cols()

i=0
Processing /home/fuzzy/DataAnalysisLab/Case_Study_1/data/202108-divvy-tripdata.csv
Columns: ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
New Columns: ID,Bike Type,Start Time,End Time,From Station Name,From Station ID,To Station Name,To Station ID,Start Latitude,Start Longitude,End Latitude,End Longitude,User Type

i=1
Processing /home/fuzzy/DataAnalysisLab/Case_Study_1/data/Divvy_Trips_2018_Q4.csv
Columns: trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
New Columns: ID,Start Time,End Time,Bike ID,Duration,From Station ID,From Station Name,To Station ID,To Station Name,User Type,Gender,Birth Year

i=2
Processing /home/fuzzy/DataAnalysisLab/Case_Study_1/data/202007-divvy-tripdata.csv
Columns: ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_n

In [6]:
FILES = list_trip_files(d=TEST_DIR)

In [7]:
len(FILES)

52

In [8]:
COLS_2_KEEP = [
               'ID', 
               'Start Time',
               'End Time',
               'Bike ID',
               'Duration',
               'From Station ID',
               'To Station ID',
               'User Type',
               'Gender',
               'Birth Year',
               'To Station Name',
               'From Station Name',
               'End Latitude',
               'End Longitude',
               'Bike Type',
               'Start Latitude',
               'Start Longitude'
              ]

In [10]:
pp(COLS_LIST)

dict_keys(['ID', 'Start Time', 'End Time', 'Bike ID', 'Duration', 'From Station ID', 'To Station ID', 'User Type', 'Gender', 'Birth Year', 'To Station Name', 'From Station Name', 'End Latitude', 'End Longitude', 'Start Latitude', 'Start Longitude', 'Bike Type'])


In [9]:
def open_data_frame(p):
    cols_2_keep = COLS_2_KEEP
#     cols_2_keep.extend([f'"{s}"' for s in cols_2_keep])
#     columnize(cols_2_keep)
    print(f'Processing file: {p.name}')
    df = pd.read_csv(str(p))
    print(f'{df.columns=}')
    MAX_LEN = len(COLS_2_KEEP)
    cols_2_drop = list(set(df.columns).difference(cols_2_keep))
    print('Columns 2 Drop: ', cols_2_drop)
    df = df.drop(columns=cols_2_drop).reindex(columns=cols_2_keep)
    if len(df.columns) > MAX_LEN:
        df.drop(df.columns[MAX_LEN:], axis=1, inplace=True)
    print('New Columns: ', df.columns)
    return df

In [10]:
df = open_data_frame(FILES[1])

Processing file: Divvy_Trips_2018_Q4.csv
df.columns=Index(['ID', 'Start Time', 'End Time', 'Bike ID', 'Duration',
       'From Station ID', 'From Station Name', 'To Station ID',
       'To Station Name', 'User Type', 'Gender', 'Birth Year'],
      dtype='object')
Columns 2 Drop:  []
New Columns:  Index(['ID', 'Start Time', 'End Time', 'Bike ID', 'Duration',
       'From Station ID', 'To Station ID', 'User Type', 'Gender', 'Birth Year',
       'To Station Name', 'From Station Name', 'End Latitude', 'End Longitude',
       'Bike Type', 'Start Latitude', 'Start Longitude'],
      dtype='object')


In [11]:
df

,ID,Start Time,End Time,Bike ID,Duration,From Station ID,To Station ID,User Type,Gender,Birth Year,To Station Name,From Station Name,End Latitude,End Longitude,Bike Type,Start Latitude,Start Longitude
0,20983530,2018-10-01 00:01:17,2018-10-01 00:29:35,4551,"1,698.0",85,166,Subscriber,Male,1992.0,Ashland Ave & Wrightwood Ave,Michigan Ave & Oak St,NaN,NaN,NaN,NaN,NaN
1,20983531,2018-10-01 00:03:59,2018-10-01 00:10:55,847,416.0,13,144,Subscriber,Female,1982.0,Larrabee St & Webster Ave,Wilton Ave & Diversey Pkwy,NaN,NaN,NaN,NaN,NaN
2,20983532,2018-10-01 00:05:14,2018-10-01 00:14:08,6188,534.0,59,197,Subscriber,Male,1986.0,Michigan Ave & Madison St,Wabash Ave & Roosevelt Rd,NaN,NaN,NaN,NaN,NaN
3,20983533,2018-10-01 00:05:48,2018-10-01 00:18:46,6372,778.0,328,419,Subscriber,Female,1960.0,Lake Park Ave & 53rd St,Ellis Ave & 58th St,NaN,NaN,NaN,NaN,NaN
4,20983534,2018-10-01 00:07:29,2018-10-01 00:25:51,1927,"1,102.0",93,159,Subscriber,Female,1993.0,Claremont Ave & Hirsch St,Sheffield Ave & Willow St,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642681,21742438,2018-12-31 23:45:17,2018-12-31 23:50:05,2931,288.0,49,164,Subscriber,Female,1983.0,Franklin St & Lake St,Dearborn St & Monroe St,NaN,NaN,NaN,NaN,NaN
642682,21742439,2018-12-31 23:48:48,2018-12-31 23:57:22,4386,514.0,624,44,Subscriber,Female,1990.0,State St & Randolph St,Dearborn St & Van Buren St (*),NaN,NaN,NaN,NaN,NaN
642683,21742440,2018-12-31 23:50:09,2018-12-31 23:57:16,4927,427.0,41,52,Subscriber,Female,1995.0,Michigan Ave & Lake St,Federal St & Polk St,NaN,NaN,NaN,NaN,NaN
642684,21742441,2018-12-31 23:55:04,2018-12-31 23:58:24,1350,200.0,141,118,Subscriber,Male,1999.0,Sedgwick St & North Ave,Clark St & Lincoln Ave,NaN,NaN,NaN,NaN,NaN


In [18]:
N = len(df)

In [19]:
len(df['ID'].unique()) == N

True

In [20]:
len(df['ID'].dropna()) == N

True

In [24]:
df.loc[N - 1, "Bike ID"] - df.loc[0, "Bike ID"] == N

False

In [26]:
len(df['Bike ID'].dropna()) == N

True

In [62]:
len(df['Bike ID'].unique())

1

In [64]:
len(df['Duration'].dropna())

0

In [29]:
ERRORS = dict()
for i in df.columns:
    ERRORS[i] = list()

In [30]:
ERR_INDEX = ["Nulls"]

In [35]:
for c in df.columns:
    ERRORS[c].append(len(df[df[c].isna()]))

In [32]:
ERRORS

{'ID': [0],
 'Start Time': [0],
 'End Time': [0],
 'Bike ID': [0],
 'Duration': [0],
 'From Station ID': [0],
 'To Station ID': [0],
 'User Type': [0],
 'Gender': [48382],
 'Birth Year': [46166],
 'To Station Name': [0],
 'From Station Name': [0],
 'End Latitude': [642686],
 'End Longitude': [642686],
 'Bike Type': [642686],
 'Start Latitude': [642686],
 'Start Longitude': [642686]}

In [33]:
errs_df = pd.DataFrame(ERRORS, index=ERR_INDEX)

In [36]:
errs_df

,ID,Start Time,End Time,Bike ID,Duration,From Station ID,To Station ID,User Type,Gender,Birth Year,To Station Name,From Station Name,End Latitude,End Longitude,Bike Type,Start Latitude,Start Longitude
Nulls,0,0,0,0,0,0,0,0,48382,46166,0,0,642686,642686,642686,642686,642686


In [29]:
stations = pd.read_csv(CLEAN_DIR / 'stations.csv')

In [32]:
help(pd.DataFrame.reindex)

Help on function reindex in module pandas.core.frame:

reindex(self, labels=None, index=None, columns=None, axis=None, method=None, copy=True, level=None, fill_value=nan, limit=None, tolerance=None)
    Conform Series/DataFrame to new index with optional filling logic.
    
    Places NA/NaN in locations having no value in the previous index. A new object
    is produced unless the new index is equivalent to the current one and
    ``copy=False``.
    
    Parameters
    ----------
    
    keywords for axes : array-like, optional
        New labels / index to conform to, should be specified using
        keywords. Preferably an Index object to avoid duplicating data.
    
    method : {None, 'backfill'/'bfill', 'pad'/'ffill', 'nearest'}
        Method to use for filling holes in reindexed DataFrame.
        Please note: this is only applicable to DataFrames/Series with a
        monotonically increasing/decreasing index.
    
        * None (default): don't fill gaps
        * pad / f

In [33]:
stations = stations.reindex(stations['ID']).drop(columns=['ID'])

In [34]:
stations

,Name,Latitude,Longitude,City,DP Capacity,Landmark,Online Date
ID,,,,,,,
2,Burnham Harbor,41.856268,-87.613348,Chicago,23.0,545.0,2015-05-16
3,State St & Harrison St,41.874029,-87.627722,Chicago,19.0,30.0,2013-06-28
4,DuSable Harbor,41.885042,-87.612795,Chicago,39.0,548.0,2015-04-24
5,Field Blvd & South Water St,41.886349,-87.617517,Chicago,19.0,534.0,2015-04-18
6,Leavitt St & Archer Ave,41.828792,-87.680604,Chicago,19.0,695.0,2015-04-14
...,...,...,...,...,...,...,...
622,NaN,NaN,NaN,NaN,NaN,NaN,NaN
623,NaN,NaN,NaN,NaN,NaN,NaN,NaN
624,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
stations.columns[0]

'ID'

In [38]:
from collections import namedtuple

In [42]:
def stations_data():
    stations = pd.read_csv(str(CLEAN_DIR / 'stations.csv'))
    stations.index = stations["ID"]
    return stations.drop(columns=["ID"])

In [43]:
stations = stations_data()

In [48]:
stations = None

In [59]:
missing_bikes = 0
START_LAT = "Start Latitude"
START_LNG = "Start Longitude"
FROM_ID = "From Station ID"
FROM_NAME = "From Station Name"

Coordinates = namedtuple('Coordinates', ["Latitude", "Longitude"])

def station_coords(i:int) -> Coordinates:
    global stations
    if stations is None:
        stations = stations_data()
    try:
        return Coordinates(stations["Latitude"][i], stations["Longitude"][i])
    except KeyError:
        print(f"Invalid station ID: {i}")
        return None

NA_START_LATS = df[START_LAT].isna()
NA_FROM_NAMES = df[FROM_NAME].isna()
NA_FROM_IDs   = df[FROM_ID].isna()

for i in df.index:
    if i in NA_START_LATS.index:
        if i in NA_FROM_IDs.index:
            from_station = df[FROM_ID][i]
        elif not i in NA_FROM_NAMES.index:
            from_station = get_station_id(df[FROM_NAME][i])
        else:
            print(f"Bad ride record, missing from station ID and name: {i}")
            break
        coords = station_coords(from_station)
        if not coords:
            print(f"Error getting station coordinates, line {i}.")
            df = df.drop(i)
            break
        df.loc[i, START_LAT] = coords.Latitude
        df.loc[i, START_LNG] = coords.Longitude

Invalid station ID: 627
Error getting station coordinates, line 147.


In [54]:
df

,ID,Start Time,End Time,Bike ID,Duration,From Station ID,To Station ID,User Type,Gender,Birth Year,To Station Name,From Station Name,End Latitude,End Longitude,Bike Type,Start Latitude,Start Longitude
0,20983530,2018-10-01 00:01:17,2018-10-01 00:29:35,4551,"1,698.0",85,166,Subscriber,Male,1992.0,Ashland Ave & Wrightwood Ave,Michigan Ave & Oak St,NaN,NaN,NaN,41.900960,-87.623777
1,20983531,2018-10-01 00:03:59,2018-10-01 00:10:55,847,416.0,13,144,Subscriber,Female,1982.0,Larrabee St & Webster Ave,Wilton Ave & Diversey Pkwy,NaN,NaN,NaN,41.932449,-87.652696
2,20983532,2018-10-01 00:05:14,2018-10-01 00:14:08,6188,534.0,59,197,Subscriber,Male,1986.0,Michigan Ave & Madison St,Wabash Ave & Roosevelt Rd,NaN,NaN,NaN,41.867220,-87.625960
3,20983533,2018-10-01 00:05:48,2018-10-01 00:18:46,6372,778.0,328,419,Subscriber,Female,1960.0,Lake Park Ave & 53rd St,Ellis Ave & 58th St,NaN,NaN,NaN,41.788746,-87.601334
4,20983534,2018-10-01 00:07:29,2018-10-01 00:25:51,1927,"1,102.0",93,159,Subscriber,Female,1993.0,Claremont Ave & Hirsch St,Sheffield Ave & Willow St,NaN,NaN,NaN,41.913688,-87.652855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642681,21742438,2018-12-31 23:45:17,2018-12-31 23:50:05,2931,288.0,49,164,Subscriber,Female,1983.0,Franklin St & Lake St,Dearborn St & Monroe St,NaN,NaN,NaN,NaN,NaN
642682,21742439,2018-12-31 23:48:48,2018-12-31 23:57:22,4386,514.0,624,44,Subscriber,Female,1990.0,State St & Randolph St,Dearborn St & Van Buren St (*),NaN,NaN,NaN,NaN,NaN
642683,21742440,2018-12-31 23:50:09,2018-12-31 23:57:16,4927,427.0,41,52,Subscriber,Female,1995.0,Michigan Ave & Lake St,Federal St & Polk St,NaN,NaN,NaN,NaN,NaN
642684,21742441,2018-12-31 23:55:04,2018-12-31 23:58:24,1350,200.0,141,118,Subscriber,Male,1999.0,Sedgwick St & North Ave,Clark St & Lincoln Ave,NaN,NaN,NaN,NaN,NaN


In [58]:
len(df[START_LAT].isna())

642684

In [94]:
df = df.dropna(subset=['To Station ID'])

In [105]:
print(f'Number of thefts: {len(df[df["To Station ID"].isna()])}')

Number of thefts: 0


In [117]:
MIN_DATE = datetime(2013, 1, 1, 0, 0, 0)
MAX_DATE = datetime(2022, 1, 31, 0, 0, 0)

In [120]:
n = 0
for t in df['End Time']:
    d = parse(t)
    if d < MIN_DATE or d > MAX_DATE:
        n += 1

In [119]:
n

0

In [ ]:
df["Duration"] = parse()

In [123]:
df.iloc[2]["Start Time"]

'2020-05-02 14:11:03'

In [130]:
for i in range(len(df)):
    df["Duration"].iloc[i] = str(parse(df["End Time"].iloc[i]) - parse(df["Start Time"].iloc[i]))

In [131]:
df

,ID,Start Time,End Time,Bike ID,Duration,From Station ID,To Station ID,User Type,Gender,Birth Year,To Station Name,From Station Name,End Latitude,End Longitude,Ridable Type,Start Latitude,Start Longitude
0,02668AD35674B983,2020-05-27 10:03:52,2020-05-27 10:16:49,docked_bike,0:12:57,36,199.0,member,NaN,NaN,Wabash Ave & Grand Ave,Franklin St & Jackson Blvd,41.8915,-87.6268,NaN,41.8777,-87.6353
1,7A50CCAF1EDDB28F,2020-05-25 10:47:11,2020-05-25 11:05:40,docked_bike,0:18:29,340,326.0,casual,NaN,NaN,Clark St & Leland Ave,Clark St & Wrightwood Ave,41.9671,-87.6674,NaN,41.9295,-87.6431
2,2FFCDFDB91FE9A52,2020-05-02 14:11:03,2020-05-02 15:48:21,docked_bike,1:37:18,260,260.0,casual,NaN,NaN,Kedzie Ave & Milwaukee Ave,Kedzie Ave & Milwaukee Ave,41.9296,-87.7079,NaN,41.9296,-87.7079
3,58991CF1DB75BA84,2020-05-02 16:25:36,2020-05-02 16:39:28,docked_bike,0:13:52,251,157.0,casual,NaN,NaN,Lake Shore Dr & Wellington Ave,Clarendon Ave & Leland Ave,41.9367,-87.6368,NaN,41.9680,-87.6500
4,A79651EFECC268CD,2020-05-29 12:49:54,2020-05-29 13:27:11,docked_bike,0:37:17,261,206.0,member,NaN,NaN,Halsted St & Archer Ave,Hermitage Ave & Polk St,41.8472,-87.6468,NaN,41.8715,-87.6699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200269,11C316914A7AF2F9,2020-05-30 12:48:07,2020-05-30 13:28:59,docked_bike,0:40:52,623,300.0,casual,NaN,NaN,Broadway & Barry Ave,Michigan Ave & 8th St,41.9377,-87.6441,NaN,41.8728,-87.6240
200270,DEA1AB8AB6A5E995,2020-05-30 20:21:32,2020-05-30 20:57:21,docked_bike,0:35:49,284,335.0,member,NaN,NaN,Calumet Ave & 35th St,Michigan Ave & Jackson Blvd,41.8314,-87.6180,NaN,41.8779,-87.6241
200271,D9EBC47F8C828B6E,2020-05-24 11:27:42,2020-05-24 11:35:20,docked_bike,0:07:38,144,343.0,casual,NaN,NaN,Racine Ave & Wrightwood Ave,Larrabee St & Webster Ave,41.9289,-87.6590,NaN,41.9218,-87.6441
200272,9744AA400432C99D,2020-05-06 17:18:46,2020-05-06 17:55:34,docked_bike,0:36:48,463,464.0,member,NaN,NaN,Damen Ave & Foster Ave,Clark St & Berwyn Ave,41.9756,-87.6795,NaN,41.9780,-87.6680


In [128]:
help(pd.Series.iloc)

Help on property:

    Purely integer-location based indexing for selection by position.
    
    ``.iloc[]`` is primarily integer position based (from ``0`` to
    ``length-1`` of the axis), but may also be used with a boolean
    array.
    
    Allowed inputs are:
    
    - An integer, e.g. ``5``.
    - A list or array of integers, e.g. ``[4, 3, 0]``.
    - A slice object with ints, e.g. ``1:7``.
    - A boolean array.
    - A ``callable`` function with one argument (the calling Series or
      DataFrame) and that returns valid output for indexing (one of the above).
      This is useful in method chains, when you don't have a reference to the
      calling object, but would like to base your selection on some value.
    
    ``.iloc`` will raise ``IndexError`` if a requested indexer is
    out-of-bounds, except *slice* indexers which allow out-of-bounds
    indexing (this conforms with python/numpy *slice* semantics).
    
    See more at :ref:`Selection by Position <indexing.inte

In [ ]:
# new_df.to_csv(str(CSV_DIR / trip_files[0].name), header=False, index=False)

In [ ]:
(CSV_DIR / 'header.csv').write_text(','.join(cols_2_keep))

In [19]:
cd(DATA_DIR)

'C:\\Users\\owner\\resume\\portfolio\\Case_Study_1\\data'

In [20]:
for p in [Path(f) for f in glob('*.csv') if f != '']:
    cols_2_keep = COLS_2_KEEP
    cols_2_keep.extend([f'"{s}"' for s in cols_2_keep])
    columnize(cols_2_keep)
    print(f'Processing file: {p.name}')
    df = pd.read_csv(str(p))
    print(f'{df.columns=}')
    cols_2_drop = list(set(df.columns).difference(cols_2_keep))
    print('Columns 2 Drop: ', cols_2_drop)
    df = df.drop(columns=cols_2_drop).reindex(columns=cols_2_keep)
    print('New Columns: ', df.columns)
    df.to_csv(str(CSV_DIR / 'header.csv'), mode='a', header=False, index=False)
    print()

Processing file: 202004-divvy-tripdata.csv
df.columns=Index(['ID', 'Bike ID', 'Start Time', 'End Time', 'From Station Name',
       'From Station ID', 'To Station Name', 'To Station ID', 'Start Latitude',
       'Start Longitude', 'End Latitude', 'End Longitude', 'User Type'],
      dtype='object')
Columns 2 Drop:  ['Start Latitude', 'End Latitude', 'Start Longitude', 'To Station Name', 'ID', 'End Longitude', 'From Station Name', 'Bike ID']
New Columns:  Index(['Start Time', 'End Time', 'From Station ID', 'To Station ID',
       'User Type', 'Gender', 'Birth Year'],
      dtype='object')

Processing file: 202005-divvy-tripdata.csv
df.columns=Index(['ID', 'Bike ID', 'Start Time', 'End Time', 'From Station Name',
       'From Station ID', 'To Station Name', 'To Station ID', 'Start Latitude',
       'Start Longitude', 'End Latitude', 'End Longitude', 'User Type'],
      dtype='object')
Columns 2 Drop:  ['Start Latitude', 'End Latitude', 'Start Longitude', 'To Station Name', 'ID', 'End Lon

In [4]:
GIANT_FILE = CSV_DIR / 'header.csv'

In [7]:
with GIANT_FILE.open() as f:
    print(f.readline())

Start Time,End Time,From Station ID,To Station ID,User Type,Gender,Birth Year,,,,,,



In [9]:
cd(CSV_DIR)
CSV_FILES = glob('*.csv')

In [10]:
CSV_FILES

['202004-divvy-tripdata.csv',
 '202005-divvy-tripdata.csv',
 '202006-divvy-tripdata.csv',
 '202007-divvy-tripdata.csv',
 '202008-divvy-tripdata.csv',
 '202009-divvy-tripdata.csv',
 '202010-divvy-tripdata.csv',
 '202011-divvy-tripdata.csv',
 '202012-divvy-tripdata.csv',
 '202101-divvy-tripdata.csv',
 '202102-divvy-tripdata.csv',
 '202103-divvy-tripdata.csv',
 '202104-divvy-tripdata.csv',
 '202105-divvy-tripdata.csv',
 '202106-divvy-tripdata.csv',
 '202107-divvy-tripdata.csv',
 '202108-divvy-tripdata.csv',
 '202109-divvy-tripdata.csv',
 '202110-divvy-tripdata.csv',
 '202111-divvy-tripdata.csv',
 '202112-divvy-tripdata.csv',
 '202201-divvy-tripdata.csv',
 'Divvy_Trips_2013.csv',
 'Divvy_Trips_2014-Q3-07.csv',
 'Divvy_Trips_2014-Q3-0809.csv',
 'Divvy_Trips_2014-Q4.csv',
 'Divvy_Trips_2014_Q1Q2.csv',
 'Divvy_Trips_2015-Q1.csv',
 'Divvy_Trips_2015-Q2.csv',
 'Divvy_Trips_2015_07.csv',
 'Divvy_Trips_2015_08.csv',
 'Divvy_Trips_2015_09.csv',
 'Divvy_Trips_2015_Q4.csv',
 'Divvy_Trips_2016_04.csv

In [13]:
OUTPUT_FILE = TEST_DIR / 'trips.csv'
ATTRS_FILE = BASE_DIR / 'header.csv'
OUTPUT_FILE.write_text(ATTRS_FILE.read_text())
print(OUTPUT_FILE.read_text())

Start Time,End Time,From Station ID,To Station ID,User Type,Gender,Birth Year


In [16]:
with Path(CSV_FILES[0]).open() as f:
    s = f.readline()
    print(s)

2020-04-26 17:45:14,2020-04-26 18:12:03,86,152.0,member,,



In [20]:
h = OUTPUT_FILE.read_text()

In [21]:
h

'Start Time,End Time,From Station ID,To Station ID,User Type,Gender,Birth Year\n'

In [19]:
OUTPUT_FILE.write_text(h + '\n')

78

In [24]:
with OUTPUT_FILE.open() as f:
    s = f.readline().rstrip('\n')
s

'Start Time,End Time,From Station ID,To Station ID,User Type,Gender,Birth Year'

In [25]:
p = Path(CSV_FILES[0])

In [26]:
p

WindowsPath('202004-divvy-tripdata.csv')

In [30]:
with p.open() as f:
    s = f.readline().rstrip('\n')

In [31]:
s

'2020-04-26 17:45:14,2020-04-26 18:12:03,86,152.0,member,,'

In [29]:
s.rstrip('\n')

'2020-04-26 17:45:14,2020-04-26 18:12:03,86,152.0,member,,'

In [33]:
len(re.compile(',').findall(s))

6

In [34]:
v = s.split(',')

In [35]:
v

['2020-04-26 17:45:14', '2020-04-26 18:12:03', '86', '152.0', 'member', '', '']

In [55]:
start, end, origin, dest, user, gender, birth = v

In [53]:
birth

''

In [56]:
start, end, origin, dest, user, gender, birth = s.split(',')
start = str(parse(start))
end = str(parse(end))
origin = int(origin.split('.')[0])
dest = int(dest.split('.')[0])
if birth:
    birth = str(parse(birth).year())


In [57]:
print(f'{start=}')
print(f'{end=}')
print(f'{origin=}')
print(f'{dest=}')
print(f'{birth=}')

start='2020-04-26 17:45:14'
end='2020-04-26 18:12:03'
origin=86
dest=152
birth=''


In [39]:
parse(start)

datetime.datetime(2020, 4, 26, 17, 45, 14)

In [40]:
str(start)

'2020-04-26 17:45:14'

In [41]:
cnx, cursor = db_connect()

Connected to database owner at 192.168.254.71


In [14]:
HEADER_FILE = BASE_DIR / 'header.csv'
SUBSCRIBER_TYPES = ['Dependent', 'Subscriber', 'member']
CUSTOMER_TYPES = ['Customer', 'casual']
UNIQUE_GENDERS = ['Female', 'Male']
HEADERS = (BASE_DIR / 'header.csv').read_text().rstrip('\n').split(',')

In [106]:
HEADERS

['Start Time',
 'End Time',
 'From Station ID',
 'To Station ID',
 'User Type',
 'Gender',
 'Birth Year']

In [110]:
p = Path(CSV_FILES[38])
with p.open() as f:
    s = f.readline().rstrip('\n')
    start, end, origin, dest, user, gender, birth = s.split(',')
    start = str(parse(start))
    end = str(parse(end))
    origin = int(origin)
    if user:
        user = 'Subscriber' if user == 'member' else ('Customer' if user == 'casual' else user)
    dest = int(dest)
    if birth:
        birth = str(int(birth.split('.')[0]))
    else:
        birth = 0
    statement = f"""INSERT INTO trips(Start_Time, End_Time, From_Station_ID, To_Station_ID, User_Type, Gender, Birth_Year) VALUES('{start}', '{end}', '{origin}', '{dest}', '{user}', '{gender}', '{birth}');"""
    cursor.execute(statement)
    cnx.commit()

In [111]:
cnx.commit()

In [ ]:
statement = f"""INSERT INTO Authors (First, Last) VALUES('{names[0]}', '{names[-1]}');"""

In [44]:
(BASE_DIR / 'header.csv').read_text().rstrip('\n')

'Start Time,End Time,From Station ID,To Station ID,User Type,Gender,Birth Year'

In [84]:
HEADERS = (BASE_DIR / 'header.csv').read_text().rstrip('\n')
HEADERS

'Start Time,End Time,From Station ID,To Station ID,User Type,Gender,Birth Year'

In [60]:
head_frame = pd.read_csv(HEADER_FILE)

In [61]:
head_frame

,Start Time,End Time,From Station ID,To Station ID,User Type,Gender,Birth Year


In [67]:
file_frame = pd.read_csv(CSV_FILES[1], header=None)

In [68]:
file_frame

,0,1,2,3,4,5,6
0,2020-05-27 10:03:52,2020-05-27 10:16:49,36,199.0,member,NaN,NaN
1,2020-05-25 10:47:11,2020-05-25 11:05:40,340,326.0,casual,NaN,NaN
2,2020-05-02 14:11:03,2020-05-02 15:48:21,260,260.0,casual,NaN,NaN
3,2020-05-02 16:25:36,2020-05-02 16:39:28,251,157.0,casual,NaN,NaN
4,2020-05-29 12:49:54,2020-05-29 13:27:11,261,206.0,member,NaN,NaN
...,...,...,...,...,...,...,...
200269,2020-05-30 12:48:07,2020-05-30 13:28:59,623,300.0,casual,NaN,NaN
200270,2020-05-30 20:21:32,2020-05-30 20:57:21,284,335.0,member,NaN,NaN
200271,2020-05-24 11:27:42,2020-05-24 11:35:20,144,343.0,casual,NaN,NaN
200272,2020-05-06 17:18:46,2020-05-06 17:55:34,463,464.0,member,NaN,NaN


In [70]:
file_frame.columns = head_frame.columns
df = pd.concat([head_frame, file_frame])
df

,Start Time,End Time,From Station ID,To Station ID,User Type,Gender,Birth Year
0,2020-05-27 10:03:52,2020-05-27 10:16:49,36,199.0,member,NaN,NaN
1,2020-05-25 10:47:11,2020-05-25 11:05:40,340,326.0,casual,NaN,NaN
2,2020-05-02 14:11:03,2020-05-02 15:48:21,260,260.0,casual,NaN,NaN
3,2020-05-02 16:25:36,2020-05-02 16:39:28,251,157.0,casual,NaN,NaN
4,2020-05-29 12:49:54,2020-05-29 13:27:11,261,206.0,member,NaN,NaN
...,...,...,...,...,...,...,...
200269,2020-05-30 12:48:07,2020-05-30 13:28:59,623,300.0,casual,NaN,NaN
200270,2020-05-30 20:21:32,2020-05-30 20:57:21,284,335.0,member,NaN,NaN
200271,2020-05-24 11:27:42,2020-05-24 11:35:20,144,343.0,casual,NaN,NaN
200272,2020-05-06 17:18:46,2020-05-06 17:55:34,463,464.0,member,NaN,NaN


In [92]:
HEADER_FILE = BASE_DIR / 'header.csv'

def read_trip_csv_frame(f):
    h = pd.read_csv(HEADER_FILE)
    ff = pd.read_csv(f)
    ff.columns = h.columns
    return pd.concat([h, ff])

def unique_values(s):
    v = list()
    for f in CSV_FILES:
        df = read_trip_csv_frame(f)
        for u in df[s].dropna().unique():
            v.append(u)
    v = list(set(v))
    v.sort()
    if '' in v:
        v = v.remove('')
    return v

In [75]:
df = read_trip_csv_frame(CSV_FILES[4])

In [76]:
df

,Start Time,End Time,From Station ID,To Station ID,User Type,Gender,Birth Year
0,2020-08-27 18:46:04,2020-08-27 19:54:51,168.0,168.0,casual,NaN,NaN
1,2020-08-26 19:44:14,2020-08-26 21:53:07,195.0,44.0,casual,NaN,NaN
2,2020-08-27 12:05:41,2020-08-27 12:53:45,81.0,47.0,casual,NaN,NaN
3,2020-08-27 16:49:02,2020-08-27 16:59:49,658.0,658.0,casual,NaN,NaN
4,2020-08-27 17:26:23,2020-08-27 18:07:50,658.0,658.0,casual,NaN,NaN
...,...,...,...,...,...,...,...
622355,2020-08-05 23:22:37,2020-08-05 23:31:33,627.0,96.0,casual,NaN,NaN
622356,2020-08-08 23:42:41,2020-08-08 23:49:00,142.0,142.0,casual,NaN,NaN
622357,2020-08-29 07:27:19,2020-08-29 07:54:26,122.0,22.0,casual,NaN,NaN
622358,2020-08-04 19:43:13,2020-08-04 21:48:11,568.0,112.0,casual,NaN,NaN


In [79]:
unique_values('User Type')

['Customer', 'Dependent', 'Subscriber', 'casual', 'member']

In [82]:
unique_values('Gender')

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 'Female', 'Male']

In [93]:
unique_values('Birth Year')

[1759.0,
 1790.0,
 1888.0,
 1889.0,
 1890.0,
 1895.0,
 1898.0,
 1899.0,
 1900.0,
 1901.0,
 1904.0,
 1905.0,
 1906.0,
 1909.0,
 1910.0,
 1911.0,
 1912.0,
 1915.0,
 1916.0,
 1918.0,
 1920.0,
 1921.0,
 1922.0,
 1923.0,
 1925.0,
 1927.0,
 1928.0,
 1929.0,
 1930.0,
 1931.0,
 1932.0,
 1933.0,
 1934.0,
 1935.0,
 1936.0,
 1937.0,
 1938.0,
 1939.0,
 1940.0,
 1941.0,
 1942.0,
 1943.0,
 1944.0,
 1945.0,
 1946.0,
 1947.0,
 1948.0,
 1949.0,
 1950.0,
 1951.0,
 1952.0,
 1953.0,
 1954.0,
 1955.0,
 1956.0,
 1957.0,
 1958.0,
 1959.0,
 1960.0,
 1961.0,
 1962.0,
 1963.0,
 1964.0,
 1965.0,
 1966.0,
 1967.0,
 1968.0,
 1969.0,
 1970.0,
 1971.0,
 1972.0,
 1973.0,
 1974.0,
 1975.0,
 1976.0,
 1977.0,
 1978.0,
 1979.0,
 1980.0,
 1981.0,
 1982.0,
 1983.0,
 1984.0,
 1985.0,
 1986.0,
 1987.0,
 1988.0,
 1989.0,
 1990.0,
 1991.0,
 1992.0,
 1993.0,
 1994.0,
 1995.0,
 1996.0,
 1997.0,
 1998.0,
 1999.0,
 2000.0,
 2001.0,
 2002.0,
 2003.0,
 2004.0,
 2005.0,
 2014.0,
 2016.0,
 2017.0]

In [3]:
STAGED_DIR = BASE_DIR / 'staged'

OUTPUT_FILE = STAGED_DIR / 'trips.csv'

In [6]:
trip_files

[WindowsPath('C:/Users/owner/resume/portfolio/Case_Study_1/data/Divvy_Trips_2015_07.csv'),
 WindowsPath('C:/Users/owner/resume/portfolio/Case_Study_1/data/202012-divvy-tripdata.csv'),
 WindowsPath('C:/Users/owner/resume/portfolio/Case_Study_1/data/202005-divvy-tripdata.csv'),
 WindowsPath('C:/Users/owner/resume/portfolio/Case_Study_1/data/202103-divvy-tripdata.csv'),
 WindowsPath('C:/Users/owner/resume/portfolio/Case_Study_1/data/Divvy_Trips_2018_Q3.csv'),
 WindowsPath('C:/Users/owner/resume/portfolio/Case_Study_1/data/202111-divvy-tripdata.csv'),
 WindowsPath('C:/Users/owner/resume/portfolio/Case_Study_1/data/202006-divvy-tripdata.csv'),
 WindowsPath('C:/Users/owner/resume/portfolio/Case_Study_1/data/Divvy_Trips_2017_Q4.csv'),
 WindowsPath('C:/Users/owner/resume/portfolio/Case_Study_1/data/202104-divvy-tripdata.csv'),
 WindowsPath('C:/Users/owner/resume/portfolio/Case_Study_1/data/Divvy_Trips_2016_06.csv'),
 WindowsPath('C:/Users/owner/resume/portfolio/Case_Study_1/data/Divvy_Trips_20

In [49]:
cd(CSV_DIR)
CSV_FILES = [Path(s) for s in glob('**')]
OUTPUT_FILE.write_text(HEADER_FILE.read_text() + '\n')
n = 0
with OUTPUT_FILE.open(mode='a+') as out:
    for p in CSV_FILES:
        print(p)
        with p.open() as f:
            while True:
                s = f.readline()
                if not s:
                    break
                print(s.rstrip('\n'), file=out)
                n += 1
print(f'{n=}')

202004-divvy-tripdata.csv
202005-divvy-tripdata.csv
202006-divvy-tripdata.csv
202007-divvy-tripdata.csv
202008-divvy-tripdata.csv
202009-divvy-tripdata.csv
202010-divvy-tripdata.csv
202011-divvy-tripdata.csv
202012-divvy-tripdata.csv
202101-divvy-tripdata.csv
202102-divvy-tripdata.csv
202103-divvy-tripdata.csv
202104-divvy-tripdata.csv
202105-divvy-tripdata.csv
202106-divvy-tripdata.csv
202107-divvy-tripdata.csv
202108-divvy-tripdata.csv
202109-divvy-tripdata.csv
202110-divvy-tripdata.csv
202111-divvy-tripdata.csv
202112-divvy-tripdata.csv
202201-divvy-tripdata.csv
Divvy_Trips_2013.csv
Divvy_Trips_2014-Q3-07.csv
Divvy_Trips_2014-Q3-0809.csv
Divvy_Trips_2014-Q4.csv
Divvy_Trips_2014_Q1Q2.csv
Divvy_Trips_2015-Q1.csv
Divvy_Trips_2015-Q2.csv
Divvy_Trips_2015_07.csv
Divvy_Trips_2015_08.csv
Divvy_Trips_2015_09.csv
Divvy_Trips_2015_Q4.csv
Divvy_Trips_2016_04.csv
Divvy_Trips_2016_05.csv
Divvy_Trips_2016_06.csv
Divvy_Trips_2016_Q1.csv
Divvy_Trips_2016_Q3.csv
Divvy_Trips_2016_Q4.csv
Divvy_Trips_2

In [27]:
Path(OUTPUT_FILE).read_text()

'Start Time,End Time,From Station ID,To Station ID,User Type,Gender,Birth Year'

In [45]:
CSV_FILES = [Path(s) for s in glob('**')]

'Start Time,End Time,From Station ID,To Station ID,User Type,Gender,Birth Year'
'2020-04-26 17:45:14,2020-04-26 18:12:03,86,152.0,member,,'
'2020-04-17 17:08:54,2020-04-17 17:17:03,503,499.0,member,,'
'2020-04-01 17:54:13,2020-04-01 18:08:36,142,255.0,member,,'
'2020-04-07 12:50:19,2020-04-07 13:02:31,216,657.0,member,,'
'2020-04-18 10:22:59,2020-04-18 11:15:54,125,323.0,casual,,'
'2020-04-30 17:55:47,2020-04-30 18:01:11,173,35.0,member,,'
'2020-04-02 14:47:19,2020-04-02 14:52:32,35,635.0,member,,'
'2020-04-07 12:22:20,2020-04-07 13:38:09,434,382.0,casual,,'
'2020-04-15 10:30:11,2020-04-15 10:35:55,627,359.0,casual,,'
'2020-04-04 15:02:28,2020-04-04 15:19:47,377,508.0,member,,'
'2020-04-04 15:22:43,2020-04-04 15:46:55,508,374.0,member,,'
'2020-04-25 15:43:52,2020-04-25 15:48:45,374,128.0,member,,'
'2020-04-24 18:09:43,2020-04-24 18:18:01,374,210.0,member,,'
'2020-04-11 17:15:19,2020-04-11 17:19:53,374,128.0,member,,'
'2020-04-20 17:18:50,2020-04-20 17:42:51,321,623.0,member,,'
'2020-04

In [51]:
(TEST_DIR / 'trips.csv').write_text(HEADER_FILE.read_text() + '\n' + CSV_FILES[12].read_text())


22465086

In [56]:
def check_trip_files():
    pp(trip_files)

In [57]:
check_trip_files()

[WindowsPath('C:/Users/owner/resume/portfolio/Case_Study_1/data/Divvy_Trips_2015_07.csv'),
 WindowsPath('C:/Users/owner/resume/portfolio/Case_Study_1/data/202012-divvy-tripdata.csv'),
 WindowsPath('C:/Users/owner/resume/portfolio/Case_Study_1/data/202005-divvy-tripdata.csv'),
 WindowsPath('C:/Users/owner/resume/portfolio/Case_Study_1/data/202103-divvy-tripdata.csv'),
 WindowsPath('C:/Users/owner/resume/portfolio/Case_Study_1/data/Divvy_Trips_2018_Q3.csv'),
 WindowsPath('C:/Users/owner/resume/portfolio/Case_Study_1/data/202111-divvy-tripdata.csv'),
 WindowsPath('C:/Users/owner/resume/portfolio/Case_Study_1/data/202006-divvy-tripdata.csv'),
 WindowsPath('C:/Users/owner/resume/portfolio/Case_Study_1/data/Divvy_Trips_2017_Q4.csv'),
 WindowsPath('C:/Users/owner/resume/portfolio/Case_Study_1/data/202104-divvy-tripdata.csv'),
 WindowsPath('C:/Users/owner/resume/portfolio/Case_Study_1/data/Divvy_Trips_2016_06.csv'),
 WindowsPath('C:/Users/owner/resume/portfolio/Case_Study_1/data/Divvy_Trips_20

In [58]:
df = pd.read_csv(CSV_FILES[12])

In [59]:
df

,2021-04-12 18:25:36,2021-04-12 18:56:55,TA1307000061,13235,member,Unnamed: 5,Unnamed: 6
0,2021-04-27 17:27:11,2021-04-27 18:31:29,KA1503000069,KA1503000069,casual,NaN,NaN
1,2021-04-03 12:42:45,2021-04-07 11:40:24,20121,20121,casual,NaN,NaN
2,2021-04-17 09:17:42,2021-04-17 09:42:48,TA1305000034,13235,member,NaN,NaN
3,2021-04-03 12:42:25,2021-04-03 14:13:42,20121,20121,casual,NaN,NaN
4,2021-04-25 18:43:18,2021-04-25 18:43:59,15542,15542,casual,NaN,NaN
...,...,...,...,...,...,...,...
337224,2021-04-09 17:09:03,2021-04-09 17:16:16,15529,13017,member,NaN,NaN
337225,2021-04-04 13:27:08,2021-04-04 14:41:11,15529,13022,casual,NaN,NaN
337226,2021-04-30 18:15:40,2021-04-30 19:12:44,15529,13022,member,NaN,NaN
337227,2021-04-18 11:40:37,2021-04-18 11:46:03,15529,13017,member,NaN,NaN


In [60]:
CSV_FILES[12].name

'202104-divvy-tripdata.csv'

In [61]:
cd(DATA_DIR)

'C:\\Users\\owner\\resume\\portfolio\\Case_Study_1\\data'

In [62]:
df = pd.read_csv(CSV_FILES[12].name)

In [63]:
df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,6C992BD37A98A63F,classic_bike,2021-04-12 18:25:36,2021-04-12 18:56:55,State St & Pearson St,TA1307000061,Southport Ave & Waveland Ave,13235,41.897448,-87.628722,41.948150,-87.663940,member
1,1E0145613A209000,docked_bike,2021-04-27 17:27:11,2021-04-27 18:31:29,Dorchester Ave & 49th St,KA1503000069,Dorchester Ave & 49th St,KA1503000069,41.805772,-87.592464,41.805772,-87.592464,casual
2,E498E15508A80BAD,docked_bike,2021-04-03 12:42:45,2021-04-07 11:40:24,Loomis Blvd & 84th St,20121,Loomis Blvd & 84th St,20121,41.741487,-87.658410,41.741487,-87.658410,casual
3,1887262AD101C604,classic_bike,2021-04-17 09:17:42,2021-04-17 09:42:48,Honore St & Division St,TA1305000034,Southport Ave & Waveland Ave,13235,41.903119,-87.673935,41.948150,-87.663940,member
4,C123548CAB2A32A5,docked_bike,2021-04-03 12:42:25,2021-04-03 14:13:42,Loomis Blvd & 84th St,20121,Loomis Blvd & 84th St,20121,41.741487,-87.658410,41.741487,-87.658410,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
337225,461A6B0728E063DB,classic_bike,2021-04-09 17:09:03,2021-04-09 17:16:16,Mies van der Rohe Way & Chestnut St,15529,Franklin St & Chicago Ave,13017,41.898587,-87.621915,41.896747,-87.635668,member
337226,CF1D3A35E3654F6F,docked_bike,2021-04-04 13:27:08,2021-04-04 14:41:11,Mies van der Rohe Way & Chestnut St,15529,Streeter Dr & Grand Ave,13022,41.898587,-87.621915,41.892278,-87.612043,casual
337227,4308ADB9171ACE49,classic_bike,2021-04-30 18:15:40,2021-04-30 19:12:44,Mies van der Rohe Way & Chestnut St,15529,Streeter Dr & Grand Ave,13022,41.898587,-87.621915,41.892278,-87.612043,member
337228,04DFB53077A17DA2,electric_bike,2021-04-18 11:40:37,2021-04-18 11:46:03,Mies van der Rohe Way & Chestnut St,15529,Franklin St & Chicago Ave,13017,41.898263,-87.622416,41.894942,-87.630438,member
